# Installing reqs

In [1]:
pip install pandas scikit-learn matplotlib seaborn wordcloud tqdm PyPDF2 gensim smart-open nltk python-dotenv langchain langchain_community openai langchain_openai chromadb langchain_huggingface pypdf 

Note: you may need to restart the kernel to use updated packages.


# Read in files and load the vector storage

In [2]:
# Needs cleaning:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import matplotlib.pyplot as plt
import seaborn as sns
import os
import glob
import openai
from tqdm import tqdm
from PyPDF2 import PdfReader
from nltk.corpus import stopwords
import string
from wordcloud import WordCloud 
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI # error? 
from langchain_openai import ChatOpenAI # sub
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS, DocArrayInMemorySearch
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_community.document_loaders.pdf import PyPDFLoader, PyPDFDirectoryLoader
from langchain_huggingface import HuggingFaceEmbeddings
from transformers import AutoTokenizer
from langchain.vectorstores import Chroma
import chromadb
import torch
import re


MODEL_NAME_KBLAB = 'KBLab/sentence-bert-swedish-cased'
MODEL_NAME_KB = 'KB/bert-base-swedish-cased'
MODEL_NAME_INTFLOAT = 'intfloat/multilingual-e5-large-instruct'

PATH_DB = './db'
COLLECTION_NAME = 'policy_collection'



"""KAILASH's DOCS"""
# Data Preparation
directory_path_eng = '/Users/kailashdejesushornig/Documents/GitHub/Policydokument/KaisProject/data/all_files_eng/no duplicates'
directory_path_swe = '/Users/kailashdejesushornig/Documents/GitHub/Policydokument/KaisProject/data/all_files_swe/no duplicates'
# documents = []  # List of text documents

# # Get all file paths from both directories
# file_paths_eng = glob.glob(os.path.join(directory_path_eng, '*.PDF'))
# file_paths_swe = glob.glob(os.path.join(directory_path_swe, '*.PDF'))
# file_paths = file_paths_eng + file_paths_swe

# # Read all PDF files from the directory with a progress bar
# for filepath in tqdm(file_paths, desc="Reading documents"):
#     # print(f"currently running: {filepath}")
#     with open(filepath, 'rb') as file:
#         reader = PdfReader(file)
#         text = ''
#         for page in reader.pages:
#             text += page.extract_text()
#         documents.append(text)
# # print(len(documents[:]))
# print(type(documents))
# 111= 32 eng, 79 (exluding 2 that result in errors)
# Two errors occurred in the swedish set: problematic files where renamed to .txt to be ignored for now. Solve later... 

"""WENDY's DOCS"""
DIR_PATH = directory_path_eng

doc_names = os.listdir(DIR_PATH)

# change '.PDF' to '.pdf'
for doc_name in doc_names:
    if doc_name.endswith('.PDF'):
        old_file_path = os.path.join(DIR_PATH, doc_name)
        new_file_name = doc_name[:-4] + '.pdf'
        new_file_path = os.path.join(DIR_PATH, new_file_name)
        os.rename(old_file_path, new_file_path)

# load pdf document. Use PyPDFDirectoryLoader for loading files in directory.
loader = PyPDFDirectoryLoader(DIR_PATH)
documents = loader.load()
documents[:1]
print(len(documents))


/Users/kailashdejesushornig/Documents/GitHub/Policydokument/KaisProject/myEnv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Ignoring wrong pointing object 6 0 (offset 0)
Ignoring wrong pointing object 8 0 (offset 0)
Ignoring wrong pointing object 10 0 (offset 0)
Ignoring wrong pointing object 12 0 (offset 0)
Ignoring wrong pointing object 14 0 (offset 0)
Ignoring wrong pointing object 16 0 (offset 0)
Ignoring wrong pointing object 22 0 (offset 0)
Ignoring wrong pointing object 24 0 (offset 0)
Ignoring wrong pointing object 26 0 (offset 0)
Ignoring wrong pointing object 34 0 (offset 0)
Ignoring wrong pointing object 36 0 (offset 0)
Ignoring wrong pointing object 42 0 (offset 0)
Ignoring wrong pointing object 47 0 (offset 0)
Ignoring wrong pointing object 57 0 (offset 0)
Ignoring wrong pointing object 6

430


In [3]:
print(len(documents))

430


# Split documents into chunks

In [4]:
def split_documents(chunk_size, documents, tokenizer_name):
    """
    Split documents into chunks of maximum size `chunk_size` tokens and return a list of documents.
    """

    # We use a hierarchical list of separators specifically tailored for splitting Markdown documents
    # This list is taken from LangChain's MarkdownTextSplitter class
    MARKDOWN_SEPARATORS = [
        "\n\n\n\n",
        "\n\n\n",
        "\n\n",
        "\n",
        ".",
        ",",
        " ",
        "",
    ]
    # Remove all whitespaces between newlines e.g. \n \n \n \n --> \n\n\n\n
    for doc in documents:
        doc.page_content = re.sub('(?<=\\n) (?=\\n)', '', doc.page_content)

    text_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
        tokenizer=AutoTokenizer.from_pretrained(tokenizer_name),
        chunk_size=chunk_size,
        chunk_overlap=chunk_size // 10,
        add_start_index=True,
        strip_whitespace=True,
        separators=MARKDOWN_SEPARATORS,
    )

    docs_processed = []
    for doc in documents:
        docs_processed += text_splitter.split_documents([doc])

    # Remove duplicates
    unique_texts = {}
    docs_processed_unique = []
    for doc in docs_processed:
        if doc.page_content not in unique_texts:
            unique_texts[doc.page_content] = True
            docs_processed_unique.append(doc)

    return docs_processed_unique

def upload_data(docs, embedding_model, chunk_size, collection_name, persist_dir):
    """
    Create a Chroma vectorstore from a list of documents.
    """
    
    # Split the documents to chunks
    docs = split_documents(
        chunk_size,  # Choose a chunk size adapted to our model
        documents,
        tokenizer_name=MODEL_NAME_KBLAB,
    )

    # # Write chunk texts to txt file
    # open('output/chunks.txt', 'w').close()
    # for chunk in docs:
    #     with open('output/chunks.txt', 'a', encoding='utf-8') as f:
    #         f.write(chunk.page_content + '\n\n')
    
    # Create Chroma DB with document chunks
    print(f"Added {len(docs)} chunks to ChromaDB")
    return Chroma.from_documents(
        documents=docs,
        embedding=embedding_model,
        collection_name=collection_name,
        persist_directory=persist_dir
    )
    
def get_embedding_model(model_name, device):
    # Initialize an instance of HuggingFaceEmbeddings with the specified parameters
    return HuggingFaceEmbeddings(
        model_name=model_name, # Provide the pre-trained model's path
        model_kwargs={'device':device}, # Pass the model configuration options
        encode_kwargs={'normalize_embeddings': True} # Set `True` for cosine similarity
    )

### Get embedding models maximimum sequence length (not strict)

In [5]:
# from sentence_transformers import SentenceTransformer
# print(f"Model's maximum sequence length: {SentenceTransformer(MODEL_NAME_KBLAB).max_seq_length}")

### Initialize embedding model

In [6]:
# Initialize embedding model
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'   # Check for CUDA enabled GPU
embedding_model = get_embedding_model(MODEL_NAME_KBLAB, device)

### Upload docuements to ChromaDB and create a vectorstore
#### Run this code if database is empty.

In [7]:
# Run this code if database is empty.
#Upload docuements to ChromaDB and create a vectorstore
vectorstore = upload_data(documents, embedding_model, 800, COLLECTION_NAME, PATH_DB)
collection = vectorstore._client.get_or_create_collection(name=COLLECTION_NAME)

Added 714 chunks to ChromaDB


### Initialize existing persisting storage

In [10]:
# Now we can load the persisted database from disk, and use it as normal.
client = chromadb.PersistentClient(path=PATH_DB)
collection = client.get_or_create_collection(name=COLLECTION_NAME)
vectorstore = Chroma(
    collection_name=COLLECTION_NAME,
    embedding_function=embedding_model,
    client=client
)
vectorstore.get()

{'ids': ['004c54f7-49ee-4c9c-96b8-2d6751d2c3e5',
  '01249579-a792-4d83-9c64-203094e059b4',
  '016513f4-1724-404a-889e-8b1ba3655063',
  '016e36a6-f7e9-4ab2-86a6-ec6dbb124422',
  '018b0e38-cc56-4261-adf0-eb17c347028c',
  '01d2c235-9988-4f8d-8c92-108badbb8ce5',
  '02c81607-598e-4deb-96f4-106e6f88cb3e',
  '02e4e2b2-fbf6-4a82-bae2-20d735bc11b4',
  '0347f08b-ba50-48fd-9cc2-0de7f204adc5',
  '039247f7-9d0a-4591-bfe3-caf7e47ab644',
  '04082b5f-a8ca-4767-9259-f47d8ddec2fe',
  '04085483-f0c1-497d-8548-444c841a6b90',
  '041cd478-342e-4dab-b589-82a58550a552',
  '044a05ed-9fdf-4f2d-8a57-c4394c2fd080',
  '04fa3ede-55b0-4b80-b853-a4d542eddf36',
  '05a3924e-b5b2-4ce1-a292-210b45ba6515',
  '06752918-bb8c-4c26-9c6b-8338955e2b0b',
  '0691c47f-de64-4869-8c1f-7ec13098d7c3',
  '06e2f969-e37c-44e2-9ecc-06e931495a5c',
  '07eab1c4-a39e-4ca5-9e21-e4d974c41c58',
  '0805327c-00af-43f2-b6b9-0efe635fbeac',
  '086d4233-45de-472a-aa13-90edf7b5ad14',
  '08c56fbe-a251-4c64-a3a9-765e61f6f553',
  '0972827c-000b-484b-b451-

# Create an agent2agent dialogue pipeline

## dummy version with meat-eating debate

In [11]:
# Import necessary libraries
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from datetime import datetime
import time
import openai
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Set OpenAI API key
openai.api_key = os.getenv('OPENAI_API_KEY')

# These agents a real chatterboxes, they need some restrains
output_length = 50

# Define system prompts for the two agents
devil_system_prompt = f"""
You are the Devil's Advocate. You will have a debate with the Angel's Advocate. Your mission is to make your case that eating meat is ethically right. 
Always meet your opponent's most recent arguement first and indicate this by writing "Reponse on opponent's arguement: ". 
Then continue by presenting a new argument to streghthen your own point of view, indicate your own view by writing "New aguments made: ".
You have a total of {output_length} words to give your response. Also, start every new sentence with a new row after a row break. 
"""

angel_system_prompt = f"""
You are the Angel's Advocate. You will have a debate with the Devil's Advocate. Your mission is to make the case that eating meat is ethically wrong. 
Always meet your opponent's most recent arguement first and indicate this by writing "\n Reponse on opponent's arguement: ". 
Then continue by presenting a new argument to streghthen your own point of view, indicate your own view by writing "\n New aguments made: ".
You have a total of {output_length} words to give your response. Also, start every new sentence with a new row after a row break. 
"""

# Initialize ChatOpenAI instances
devil_agent = ChatOpenAI(api_key=openai.api_key, model="gpt-4o")
angel_agent = ChatOpenAI(api_key=openai.api_key, model="gpt-4o")

# Define initial task prompt for the devil agent
task_prompt = "Discuss how eating meat is ethically right or wrong. You will start by making presenting your point of view on the matter."

# Function to create a debate between the two agents
def run_debate(devil_prompt, angel_prompt, task_prompt, num_rounds=3):
    devil_message = task_prompt
    dialogue = []

    for round_num in range(num_rounds):
        # Devil agent's turn
        devil_response = devil_agent.invoke([{"role": "system", "content": devil_prompt}, {"role": "user", "content": devil_message}])
        devil_message = devil_response.content
        dialogue.append(f"\n \n #####Devil: {devil_message}")
        
        # Angel agent's turn
        angel_response = angel_agent.invoke([{"role": "system", "content": angel_prompt}, {"role": "user", "content": devil_message}])
        angel_message = angel_response.content
        dialogue.append(f"\n \n #####Angel: {angel_message}")
        
        # Prepare for the next round
        devil_message = angel_message

    return dialogue

# Start the timer
start_time = time.time()
# Run the debate
debate_dialogue = run_debate(devil_system_prompt, angel_system_prompt, task_prompt, num_rounds=2)

# Verbose: Print the full dialogue
# for line in debate_dialogue:
    # print(line)

# End the timer
end_time = time.time()
elapsed_time = end_time - start_time
# Save the dialogue to a .txt file
current_time = datetime.now().strftime("%Y%m%d_%H%M%S")
file_name = f"debate_dialogue_{current_time}.txt"
with open(file_name, "w") as file:
    for line in debate_dialogue:
        file.write(line + "\n")
    file.write(f"\nTime taken: {elapsed_time:.2f} seconds\n")

print(f"Debate dialogue saved to {file_name}")
print(f"Time taken: {elapsed_time:.2f} seconds")

# Could be extended by another LLM analysing an long debate script and summarize it. 

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Debate dialogue saved to debate_dialogue_20240717_100511.txt
Time taken: 7.91 seconds


## Full version for loopholes [Work in progress]

In [12]:
# Import necessary libraries
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from datetime import datetime
import time
import openai
import os
from dotenv import load_dotenv
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_community.callbacks import get_openai_callback


# Load environment variables from .env file
load_dotenv()
openai.api_key = os.getenv('OPENAI_API_KEY')

# These agents a real chatterboxes, they need some restrains
output_length = 50

# Define system prompts for the two agents
devil_system_prompt = f"""
You are the Devil's Advocate. You will have a debate with the Angel's Advocate. Your mission is to find and exploit loopholes in the university's alcohol policy, with the goal to drink a beer during work hours. 
You must build your arguments on the information given in the documents you have from the vector storage. 
Find the relevant document for the matter, make a point based on the information given in it and reference the source by stating a quote from the documents.

Always meet your opponent's most recent arguement first and indicate this by writing "Reponse on opponent's arguement: ". 
Then continue by presenting a new argument to streghthen your own point of view, indicate your own view by writing "New aguments made: ".
You have a total of {output_length} words to give your response. Also, start every new sentence with a new row after a row break. 
"""

angel_system_prompt = f"""
You are the Angel's Advocate. You will have a debate with the Devil's Advocate. Your mission is to prevent the exploitation of loopholes in the university's alcohol policy. 
You must build your arguments on the information given in the documents you have from the vector storage. 
Find the relevant document for the matter, make a point based on the information given in it and reference the source by stating a quote from the documents.

Always meet your opponent's most recent arguement first and indicate this by writing "\n Reponse on opponent's arguement: ". 
Then continue by presenting a new argument to streghthen your own point of view, indicate your own view by writing "\n New aguments made: ".
You have a total of {output_length} words to give your response. Also, start every new sentence with a new row after a row break. 
"""

# Initialize ChatOpenAI instances
devil_agent = ChatOpenAI(api_key=openai.api_key, model="gpt-4o")
angel_agent = ChatOpenAI(api_key=openai.api_key, model="gpt-4o")

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()

# Define initial task prompt for the devil agent
task_prompt = "Discuss why the policy does allow you to drink beer during working hours. You will start by making presenting your point of view on the matter."

# Function to create a debate between the two agents
def run_debate(devil_prompt, angel_prompt, task_prompt, num_rounds=3):
    devil_message = task_prompt
    dialogue = []

    for round_num in range(num_rounds):

        def format_docs(docs):
            return "\n\n".join(doc.page_content for doc in docs)

        # Devil agent's turn
        rag_chain = (
            {"context": retriever | format_docs, "question": RunnablePassthrough()} # question here becomes the input for debate
            | devil_system_prompt
            | devil_agent
            | StrOutputParser()
        )
        devil_response = rag_chain.invoke(devil_message)
        devil_message = devil_response.content
        dialogue.append(f"\n \n #####Devil: {devil_message}")
        
        
        # Angel agent's turn
        rag_chain = (
            {"context": retriever | format_docs, "question": RunnablePassthrough()}
            | angel_system_prompt
            | angel_agent
            | StrOutputParser()
        )
        angel_response = angel_agent.invoke(devil_message)
        angel_message = angel_response.content
        dialogue.append(f"\n \n #####Angel: {angel_message}")
        
        # Prepare for the next round
        devil_message = angel_message

    return dialogue

# Start the timer
start_time = time.time()

# Run the debate
debate_dialogue = run_debate(devil_system_prompt, angel_system_prompt, task_prompt, num_rounds=2)

# Verbose: Print the full dialogue
# for line in debate_dialogue:
    # print(line)

# End the timer
end_time = time.time()
elapsed_time = end_time - start_time

# Save the dialogue to a .txt file
current_time = datetime.now().strftime("%Y%m%d_%H%M%S")
file_name = f"debate_dialogue_{current_time}.txt"
with open(file_name, "w") as file:
    for line in debate_dialogue:
        file.write(line + "\n")
    file.write(f"\nTime taken: {elapsed_time:.2f} seconds\n")

print(f"Debate dialogue saved to {file_name}")
print(f"Time taken: {elapsed_time:.2f} seconds")

# Could be extended by another LLM analysing an long debate script and summarize it. 

TypeError: unsupported operand type(s) for |: 'dict' and 'str'